In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
names = 'Class,Alcohol,Malic Acid,Ash,Alcalinity of ash,Magnesium,Total Phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline'
names = names.split(',')
df = pd.read_csv('./../wine/wine.data', names=names)

In [3]:
df2 = pd.DataFrame()

In [4]:
for col in df:
    if df[col].dtype == 'float64':
        if np.abs(df[col].skew()) > 0.05:
            df2[col] = stats.boxcox(df[col])[0]
        else:
            df2[col] = df[col]
        

In [5]:
df2

,Alcohol,Malic Acid,Ash,Alcalinity of ash,Total Phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines
0,21.927545,0.498435,1.787608,8.226937,1.496316,3.06,-1.078493,1.057792,1.848571,1.04,4.035440
1,19.868336,0.532808,1.374804,6.245109,1.386825,2.76,-1.130654,0.264986,1.563014,1.05,3.188416
2,19.789201,0.763944,2.145671,9.478776,1.496316,3.24,-1.028992,1.405015,1.856632,1.03,2.828647
3,22.210560,0.609549,1.890549,8.735865,2.217519,3.49,-1.185841,0.980108,2.222884,0.86,3.267868
4,19.947535,0.836270,2.454813,10.435768,1.496316,2.69,-0.832162,0.713148,1.547593,1.04,2.463578
...,...,...,...,...,...,...,...,...,...,...,...
173,20.882829,1.374665,1.816892,10.239299,0.623098,0.61,-0.599815,0.059243,2.207811,0.64,0.830624
174,20.264962,1.135061,1.861009,11.207713,0.723946,0.75,-0.755304,0.379369,2.145653,0.70,0.613335
175,20.006976,1.196201,1.542896,10.041348,0.545971,0.69,-0.755304,0.327170,2.539616,0.59,0.613335
176,19.808979,0.836270,1.700374,10.041348,0.597537,0.68,-0.583789,0.422139,2.429825,0.60,0.684770


In [43]:
mean_list = []
variance_list = []

for col in df2:
    mean = df2[col].mean()
    mean_list.append(mean)
    sum = 0
    for x in df2[col]:
        sum += (x - mean)**2
    variance = sum/(df2[col].shape[0] - 1)
    # df2[col] = [(x-mean)/np.sqrt(variance) for x in df2[col]]
    # mean_list.append(df2[col].mean())
    # sum = 0
    # for x in df2[col]:
    #     sum += (x - mean)**2
    # variance = sum/(df2[col].shape[0] - 1)
    variance_list.append(variance)

In [44]:
len(mean_list)

11

In [45]:
len(variance_list)

11

In [51]:
mean_confidence_interval_list = []
freedom = df2.shape[0] - 1
alpha = 0.05
t_val_lower = stats.t.ppf(alpha/2, freedom)
t_val_upper = stats.t.ppf(1 - alpha/2, freedom)
# chi_lower = stats.chi2.ppf()

for i in range(len(mean_list)):
    lower_bound = mean_list[i] + t_val_lower*np.sqrt(variance_list[i]/(freedom + 1))
    upper_bound = mean_list[i] + t_val_upper*np.sqrt(variance_list[i]/(freedom + 1))
    mean_confidence_interval_list.append((lower_bound, upper_bound))




In [58]:
mean_intervals = pd.DataFrame(np.array(mean_confidence_interval_list), index = df2.columns, columns=['lower_bound', 'upper_bound'])

In [59]:
mean_intervals

,lower_bound,upper_bound
Alcohol,19.251127,19.724238
Malic Acid,0.596746,0.704131
Ash,1.646876,1.763501
Alcalinity of ash,9.606118,10.002682
Total Phenols,1.026565,1.170418
Flavanoids,1.881522,2.177018
Nonflavanoid phenols,-0.962112,-0.883946
Proanthocyanins,0.423836,0.563691
Color intensity,1.541562,1.694218
Hue,0.923640,0.991259


In [53]:
chi2_lower = stats.chi2.ppf(0.025, freedom)

In [54]:
chi2_upper = stats.chi2.ppf(0.975, freedom)

In [56]:
variance_confidence_interval_list = []

for i in range(len(variance_list)):
    lower_bound = (freedom * variance_list[i])/chi2_upper
    upper_bound = (freedom * variance_list[i])/chi2_lower
    variance_confidence_interval_list.append((lower_bound, upper_bound))

In [60]:
variance_intervals = pd.DataFrame(np.array(variance_confidence_interval_list), index = df2.columns, columns=['lower_bound', 'upper_bound'])
variance_intervals


,lower_bound,upper_bound
Alcohol,2.098405,3.186808
Malic Acid,0.108106,0.164178
Ash,0.127511,0.193648
Alcalinity of ash,1.474307,2.239002
Total Phenols,0.194001,0.294625
Flavanoids,0.818588,1.243174
Nonflavanoid phenols,0.057280,0.086990
Proanthocyanins,0.183367,0.278477
Color intensity,0.218470,0.331786
Hue,0.042865,0.065098
